In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# -------------------------------
#     Loading Your Detections
# -------------------------------
#  Use the variables below to point the Toolbox scripts at your data.
#  Fill in the strings that are commented with # REQUIRED:  
#  then highlight a cell and hit Run Cell ('play' button above)

from common_python.load_detections import loadDetections
from common_python.filter_detections import filterDetections

#  Set some variables
detection_file = ''      #  REQUIRED: And provide the filename here


version_id = '' #  OPTIONAL: Manually set a version ID number [0 for initial(first) load]
time_interval = 60 #  OPTIONAL: Time interval used to evaluate suspect detections (in minutes)

#  OPTIONAL: Average Distance from receivers, in meters, at which tags can be detected
detection_radius = '' 
#  There will be an opportunity later to change these values for individual stations
#  If entered: Value must be between 0 and 999 


#  Create a suspect detections file?
SuspectDetections = True # True / False 

#  If you'd like to create a customizable distance matrix file for adjusting the distance 
#  between stations found in the above detection file
CreateDistanceMatrix = True # True / False

#  If you want to reload the input file into the internal database
#  Setting this to False will match a previously loaded instance of this filename and 
#  use the data from there. For very large and unchanging input datasets, it's OK
#  to set this to False.
ReloadInputFile = True  # True / False

#  Loads detection data into the PostgreSQL table and returns a list of table names for the tables it creates
table_names = loadDetections(detection_file, version_id, CreateDistanceMatrix, 
                   ReloadInputFile, SuspectDetections, time_interval, detection_radius) 

# Wait for output below to say 'Loading complete.'

In [ ]:
# -----------------------------------------------
#     Filtering Detections on Distance / Time
#
# -----------------------------------------------

# Tables created by this script and the script above are returned by 
# loadDetections() and listed in the 'table_names' variable.
# Types of table are Detection, station matrix, and suspect 


# If you have your own file of suspect detections, point to it here 
# and set OverrideSuspectDetectionFile to 'True':
OverrideSuspectDetectionFile = None
# Can replace the generated Suspect Detections file with your hand-edited file of suspect detections
SuspectFile = None # '/home/sandbox/data/sandbox/My_Suspect_Detections_File.csv' 

filterDetections(detection_file, version_id, SuspectFile, 
                     OverrideSuspectDetectionFile, CreateDistanceMatrix, 
                     detection_radius, ReloadInputFile)  # Wait for Out[] to say 'Filtering complete.'

In [ ]:
# ---------------------
#     Interval Data 
# ---------------------

# Creates a compressed detections table and an interval data file
# Intervals are lengths of time in which a station detected an animal.
# Many consecutive detections of an animal are replaced by one interval

import common_python.interval_data_tool as inter_data

dist_matrix_file = "%s_distance_matrix_%s.%s" % (detection_file.split('.')[0], table_names['detections'].split('_')[-1], detection_file.split('.')[1])
# Or supply your own custom dist_matrix_filename here.
# dist_matrix_file = "my_custom_distance_matrix_filename.csv"

inter_data.intervalData(detection_file, dist_matrix_file)

In [ ]:
# ---------------------------
#    Distance Matrix Merge
# ---------------------------

# Merge two distance matrix files together. Useful for overriding distances 
# between stations that are not straight-line distances (ie river systems)

import common_python.dis_mtrx_merge as dis

# Merge two Distance Matrix files, letting the duplicate pair-distance values 

dist_matrix_file = "%s_distance_matrix_v00.%s" % tuple(detection_file.split('.')) # the full file that you had before
# dist_matrix_file = 'my_custom_distance_matrix_filename.csv'

# The file that has only the stations that require an overridden value for the distance between them 
# (in cases of stations with land or other obstacles between them which make a straight-line distance incorrect.
distance_real_input = 'your_manually_edited_distance_matrix.csv'


dis.dis_mtx_merge('reqmerge', dist_matrix_file, distance_real_input)